In [1]:
# If executing in Google Colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
path_1 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/word2vec_result_final_500.pickle"
path_2 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_result_final_500.pickle"


data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

In [0]:
data.shape

In [0]:
X_pred.shape

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

y = data.importance         
target = data.importance
X = data.drop(['importance', 'doc_id'], axis=1, inplace=False)     # taking the training data features
                        
# del data
data.drop(['importance', 'doc_id'], axis=1, inplace=True) 

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

# y_train = y_train.append(y_train[y_train == 1], ignore_index=True)
# X_train = X_train.append(X_train[y_train == 1], ignore_index=True)

del data, target
gc.collect()

In [0]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [8]:
y_train = y_train.append(y_train[y_train == 1], ignore_index=True)
X_train = X_train.append(X_train[y_train == 1], ignore_index=True)

ValueError: ignored

In [9]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(449603, 500) (567226,) (192687, 500) (192687,)


### Base Model

In [0]:
base_model = XGBClassifier(objective = 'binary:logistic', tree_method = 'gpu_hist')

base_model.fit(X_train, y_train)
f1_score(base_model.predict(X_test), y_test)

0.23199552969792592

### Tuned Model

In [0]:
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(objective = 'binary:logistic', tree_method = 'gpu_hist')
param_dist = {'n_estimators': stats.randint(20, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8]
             }
clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 100, scoring = 'roc_auc', error_score = 0, verbose = 1, cv = 3)
best_model = clf.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


TypeError: ignored

In [0]:
best_model.best_estimator_.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.6810181620011485,
 'gamma': 0,
 'learning_rate': 0.4798402558421014,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 3,
 'missing': None,
 'n_estimators': 267,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 0.7103102301682198,
 'tree_method': 'gpu_hist',
 'verbosity': 1}

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# model = XGBClassifier(tree_method = 'gpu_hist', n_estimators = best_model.best_estimator_.get_params()['n_estimators'], 
#                       learning_rate = best_model.best_estimator_.get_params()['learning_rate'],
#                       subsample = best_model.best_estimator_.get_params()['subsample'],
#                       max_depth = best_model.best_estimator_.get_params()['max_depth'],
#                       colsample_bytree = best_model.best_estimator_.get_params()['colsample_bytree'],
#                       min_child_weight = best_model.best_estimator_.get_params()['min_child_weight']
#                       )


model = XGBClassifier(tree_method = 'gpu_hist', n_estimators = 267, 
                      learning_rate = 0.4798402558421014,
                      subsample = 0.7103102301682198,
                      max_depth = 7,
                      colsample_bytree = 0.6810181620011485,
                      min_child_weight = 3)

import time

s = time.time()
# scores = cross_val_score(model, X_train, y_train, cv=5)
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

Model trained in 0.2756046454111735 min
0.7563146449942134
0.4750640029514025


In [0]:
np.average(scores)

0.7522636673547493

In [0]:
print(f1_score(model.predict(X_train), y_train))

0.8151223696457577


In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []
acc = []
f1_train = []

pred = (model.predict_proba(X_test))[:,1]
pred_train = (model.predict_proba(X_train))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))
    f1_train.append(f1_score(pred_train >= i, y_train))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cutoff, y = f1, mode='lines', name='f1'))
fig.add_trace(go.Scatter(x = cutoff, y = acc, mode='lines', name='accuracy'))
fig.add_trace(go.Scatter(x = cutoff, y = f1_train, mode='lines', name='f1 train'))
fig.show()

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_sent.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = model.predict(X_pred)
probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result